In [1]:
import numpy as np
import pandas as pd
from pylab import plot,show,hist,bar
%config IPCompleter.greedy=True


In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_csv("../data/train.csv",sep="\t")
#data = pd.read_csv("../data/test.csv",sep="\t")
print len(data)
#data = data.sample#(frac=0.5)
print len(data)

1482535
1482535


# Overall description 

In [92]:
print data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 15 columns):
train_id             1482535 non-null int64
name                 1482535 non-null object
item_condition_id    1482535 non-null int64
category_name        1476208 non-null object
brand_name           849853 non-null object
price                1482535 non-null float64
shipping             1482535 non-null int64
item_description     1482531 non-null object
log_price            1482535 non-null float64
Cat1                 1482535 non-null object
Cat2                 1482535 non-null object
Cat3                 1482535 non-null object
mean_Cat1            1482535 non-null float64
mean_Cat2            1482535 non-null float64
mean_Cat3            1482535 non-null float64
dtypes: float64(5), int64(3), object(7)
memory usage: 130.1+ MB
None


,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,log_price,Cat1,Cat2,Cat3,mean_Cat1,mean_Cat2,mean_Cat3
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,2.397895,Men,Tops,T-shirts,3.228676,2.799998,2.771819
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,3.970292,Electronics,Computers & Tablets,Components & Parts,2.986254,3.810479,3.336755
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,2.397895,Women,Tops & Blouses,Blouse,3.080709,2.777419,2.647790
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,3.583519,Home,Home Décor,Home Décor Accents,2.990723,2.893886,2.908575
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,3.806662,Women,Jewelry,Necklaces,3.080709,2.829452,2.729020


### Observations
1. Few category names missing
2. 42% of brand_name in missing 1
3. category_name , brand_name , item_description have some missing values

In [ ]:
print data.head()

In [ ]:
data.describe()

In [ ]:
data.describe(include=['O'])

In [ ]:
hist(x=data[data.price<300].price,bins=100,normed=True)
show()

In [ ]:
hist(x=np.log(data.price+1),normed=True)
show()


In [4]:
data['log_price'] = np.log(data.price+1)


### observation
1. Price is highly skewed
2. Log price looks good

# Item_condition_id

In [ ]:
print data.item_condition_id.unique()

In [ ]:
hist(data.item_condition_id)
show()

In [ ]:
data[['log_price','item_condition_id']].groupby('item_condition_id').describe()

In [41]:
#data['median_item_condition_id']=data.item_condition_id.map({1:18.0,2:17.0,3:16.0,4:15.0,5:19.0})
#print data.item_condition_id

it_id_rv=data[['log_price','item_condition_id']].groupby(['item_condition_id']).describe()
it_id_rv_dict = dict(zip(it_id_rv['log_price']['mean'].index,it_id_rv['log_price']['mean']))
data['mean_it_id']=map(lambda x:it_id_rv_dict[x],data.item_condition_id)

### observation 
1. 5 condition is really low , but has the highest median price ( maybe means good condition )
2. 4 count is low , even its median price is low ( maybe means really bad condition )

In [ ]:
hist(np.log(data[data.item_condition_id==1].price+1))
show()

In [ ]:
hist(np.log(data[data.item_condition_id==2].price+1))
show()

In [ ]:
hist(np.log(data[data.item_condition_id==3].price+1))
show()

In [ ]:
hist(np.log(data[data.item_condition_id==4].price+1))
show()

In [ ]:
hist(np.log(data[data.item_condition_id==5].price+1))
show()

###  observation 
1. from log price plot we see , that almost for all categories the distribution is same except that it is somewhat different for 5th

# Category_name  





In [ ]:
print data.category_name[:10]

In [ ]:
print len(data.category_name.unique())

In [5]:
data['Cat1']=data.category_name.str.split('/').str.get(0)
data['Cat2']=data.category_name.str.split('/').str.get(1)
data['Cat3']=data.category_name.str.split('/').str.get(2)
data.Cat1= data.Cat1.fillna('NotPresent')
data.Cat2= data.Cat2.fillna('NotPresent')
data.Cat3= data.Cat3.fillna('NotPresent')

In [6]:
cat1_rv=data[['log_price','Cat1']].groupby(['Cat1']).describe()
cat1_rv_dict = dict(zip(cat1_rv['log_price']['mean'].index,cat1_rv['log_price']['mean']))
data['mean_Cat1']=map(lambda x:cat1_rv_dict[x],data.Cat1)
#print data.mean_Cat1

In [7]:
cat2_rv=data[['log_price','Cat2']].groupby(['Cat2']).describe()
cat2_rv_dict = dict(zip(cat2_rv['log_price']['mean'].index,cat2_rv['log_price']['mean']))
data['mean_Cat2']=map(lambda x:cat2_rv_dict[x],data.Cat2)

In [8]:
cat3_rv=data[['log_price','Cat3']].groupby(['Cat3']).describe()
cat3_rv_dict = dict(zip(cat3_rv['log_price']['mean'].index,cat3_rv['log_price']['mean']))
data['mean_Cat3']=map(lambda x:cat3_rv_dict[x],data.Cat3)

In [ ]:
data=data.drop(['Cat1','Cat2','Cat3','category_name'],axis=1)

In [ ]:
print data.head()

### Cat1

In [6]:
len(data.Cat1.unique())

11

In [ ]:
plot(xrange(10),data[['price','Cat1']].groupby(['Cat1']).median(),'-x')
show()

In [ ]:
data[['price','Cat1']].groupby(['Cat1']).describe()#.sort_values(by='count', ascending=False)

In [ ]:
hist(np.log(data[data['Cat1']=='Men'].price+1))
show()

####  likewise we could find out the fit for other 11 unique cat1 types

####  cat2

In [7]:
len(data.Cat2.unique())

114

In [96]:
data[['price','Cat2']].groupby(['Cat2']).describe()

MemoryError: 

In [ ]:
hist(np.log(data[data['Cat2']=='Toy'].price+1))
show()

#### find out others

####  cat3

In [8]:
len(data.Cat3.unique())

871

In [ ]:
plot(xrange(870),data[['price','Cat3']].groupby(['Cat3']).median())

.

##  shipping


In [6]:
data[['shipping','log_price']].groupby(['shipping']).describe()

log_price                                                         \
             count      mean       std  min       25%       50%       75%   
shipping                                                                    
0         819435.0  3.133894  0.693802  0.0  2.639057  3.044522  3.526361   
1         663100.0  2.787720  0.770638  0.0  2.197225  2.708050  3.258097   

                    
               max  
shipping            
0         7.605890  
1         7.601402

In [4]:
data[['shipping','price']].groupby(['shipping']).describe()

price                                                    
             count       mean       std  min   25%   50%   75%     max
shipping                                                              
0         819435.0  30.111778  39.53592  0.0  13.0  20.0  33.0  2009.0
1         663100.0  22.567726  36.95579  0.0   8.0  14.0  25.0  2000.0

In [ ]:
hist(np.log(data[data.shipping==1].price+1))

In [55]:
sp_rv=data[['log_price','shipping']].groupby(['shipping']).describe()
sp_rv_dict = dict(zip(sp_rv['log_price']['mean'].index,sp_rv['log_price']['mean']))
data['mean_shipping']=map(lambda x:sp_rv_dict[x],data.shipping)

In [56]:
train,test = train_test_split(data,test_size=0.3)

In [63]:
test.price.values

array([  12.,  102.,   10., ...,   24.,   20.,   19.])

In [64]:
import sys
print(sys.version)

2.7.13 (v2.7.13:a06454b1afa1, Dec 17 2016, 20:42:59) [MSC v.1500 32 bit (Intel)]


In [59]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(train[['shipping']],train['log_price'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [60]:
preds=lr.predict(test[['shipping']])
np.sqrt(np.mean(np.square(preds-test['log_price'])))

0.72946764650975071

##  name

In [46]:
print 1.0*len(data.name.unique())
print len(data)

1225273.0
1482535


In [12]:
data['len_name']=list(map(lambda x : len(x),data.name))

In [ ]:
data.len_name.mean()

In [ ]:
plot(xrange(len(data[['len_name','price']].groupby(['len_name']).count().price)),data[['len_name','price']].groupby(['len_name']).count().price)

In [ ]:
plot(xrange(79),data[['len_name','price']].groupby(['len_name']).median())
show()

In [ ]:
hist(np.log(data[data['len_name']==23].price+1))
show()

## brand_name

In [ ]:
1.0*len(data[data.brand_name.isnull()==True])/len(data)

In [ ]:
len(data.brand_name.unique())

In [94]:
data[data.brand_name.isnull()==True].describe(include=['O'])

,name,category_name,brand_name,item_description,Cat1,Cat2,Cat3
count,632682,629225,0.0,632679,632682,632682,632682
unique,550973,1270,0.0,533974,11,114,859
top,Bundle,"Women/Athletic Apparel/Pants, Tights, Leggings",NaN,No description yet,Women,Makeup,Other
freq,1226,23721,NaN,37373,210842,49199,31574


In [ ]:
data[data.brand_name.isnull()==True].describe()

In [78]:
data.brand_name= data.brand_name.fillna('NotPresent')

In [79]:
brand_name_rv=data[['brand_name','log_price']].groupby(['brand_name']).describe()
brand_name_rv_dict = dict(zip(brand_name_rv['log_price']['mean'].index,brand_name_rv['log_price']['mean']))
data['mean_brand_name']=map(lambda x:brand_name_rv_dict[x],data.brand_name)

In [84]:
print brand_name_rv_dict['Channel']


3.61996629566


##  item_description 


In [4]:
data.item_description=data.item_description.fillna('No description yet')

In [15]:
print data.item_description.describe()
data.price.iloc[1]

count                 741268
unique                650600
top       No description yet
freq                   41445
Name: item_description, dtype: object


24.0

In [30]:
data[(data.item_description=='No description yet') & (data.brand_name.isnull()==True)].describe(include=['O'])

,name,category_name,brand_name,item_description
count,18755,18558,0.0,18755
unique,18375,776,0.0,1
top,Bundle,Women/Tops & Blouses/T-Shirts,NaN,No description yet
freq,47,755,NaN,18755


In [31]:
data[(data.item_description=='No description yet') & (data.brand_name.isnull()==True)].category_name

1395350                          Men/Men's Accessories/Belts
142569     Electronics/Cell Phones & Accessories/Cases, C...
238563                                     Women/Shoes/Boots
771303                               Women/Jewelry/Necklaces
838205                                     Other/Other/Other
18849                      Women/Women's Accessories/Wallets
156116                                    Men/Shoes/Athletic
116639                        Kids/Toys/Electronics for Kids
1288413                        Kids/Boys (4+)/Top & T-shirts
44521                                  Women/Sweaters/Hooded
1124272              Women/Athletic Apparel/Snowsuits & Bibs
1198476                              Home/Home Décor/Baskets
962443                         Women/Tops & Blouses/T-Shirts
479189                                 Electronics/Media/DVD
463039                         Home/Seasonal Décor/Halloween
229862                          Handmade/Paper Goods/Sticker
1135579    Electronics/C

In [ ]:
len_desc = list(map(lambda x:len(x),data.item_description))
len_desc

In [86]:
i=12
print data[data.price>1500].price.iloc[i]
print data[data.price>1500].brand_name.iloc[i]
print data[data.price>1500].item_description.iloc[i]
print data[data.price>1500].name.iloc[i]

1600.0
Chanel
On hold dont purchase. ..Gorgeous like new authentic chanel boy bag. Genuine lambskin in beautiful condition, color super black very glossy. Made in Italy. Has been worn very gently. Inside is clean with one pocket. Medium size 10x6x3 quite roomy for your essentials. Long metal strap in great shape. All corners, bottom, front, back and sides are great. Very minor invisible scratch underneath the flap. It comes with dust bag, chanel box, chanel ribbon, care booklet and authenticity card. Ready to wear. Final sale. Thanks.
Lovely Auth. Chanel Boy lambskin Italy


In [93]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
 
data = "All work and no play makes jack dull boy. All work and no play makes jack a dull boy."
stopWords = set(stopwords.words('english'))
words = word_tokenize(data)
wordsFiltered = []
 
for w in words:
    if w not in stopWords:
        wordsFiltered.append(w)
 
print(wordsFiltered)

ImportError: No module named nltk.tokenize

# Pairwise


In [40]:
aa = data[['log_price','brand_name','Cat1','Cat2','Cat3','item_condition_id']].groupby(['brand_name','item_condition_id','Cat1','Cat2','Cat3']).mean().sort_values(['log_price'],ascending=False)['log_price'].to_dict()

In [62]:
data['letssee']=list(map(lambda x:aa[(x.brand_name,x.Cat1,x.Cat2,x.Cat3,x.item_condition_id)],data))

AttributeError: 'str' object has no attribute 'brand_name'

In [91]:
map(lambda y:y[1][1],data.iloc[:10].iterrows())

['MLB Cincinnati Reds T Shirt Size XL',
 'Razer BlackWidow Chroma Keyboard',
 'AVA-VIV Blouse',
 'Leather Horse Statues',
 '24K GOLD plated rose',
 'Bundled items requested for Ruie',
 'Acacia pacific tides santorini top',
 'Girls cheer and tumbling bundle of 7',
 'Girls Nike Pro shorts',
 'Porcelain clown doll checker pants VTG']

In [84]:
data.iteritems()[0]

TypeError: 'generator' object has no attribute '__getitem__'


##  Prediction 

In [ ]:
print data.columns

In [ ]:
data.Cat1= data.Cat1.fillna('NotPresent')
le = LabelEncoder()
le.fit(data.Cat1)
print len(le.classes_)
data['Cat1Tf'] = le.transform(data.Cat1) 


In [ ]:
data.Cat2= data.Cat2.fillna('NotPresent')
le = LabelEncoder()
le.fit(data.Cat2)
print len(le.classes_)
data['Cat2Tf'] = le.transform(data.Cat2) 


In [ ]:
data.Cat3= data.Cat3.fillna('NotPresent')
le = LabelEncoder()
le.fit(data.Cat3)
print len(le.classes_)
data['Cat3Tf'] = le.transform(data.Cat3) 


In [ ]:
data.brand_name=data.brand_name.fillna('OTHERS')
le=LabelEncoder()
le.fit(data.brand_name)
print len(le.classes_)
data['brand_nameTf']= le.transform(data.brand_name) 


In [ ]:
data['log_price'] = np.log(data.price+1)


In [17]:
print data.columns

Index([u'train_id', u'name', u'item_condition_id', u'brand_name', u'price',
       u'shipping', u'item_description', u'log_price', u'mean_it_id',
       u'mean_Cat1', u'mean_Cat2', u'mean_Cat3', u'mean_shipping', u'len_name',
       u'mean_brand_name'],
      dtype='object')


In [17]:
data1 = data.drop(['price',u'Cat1',u'Cat2', u'Cat3','train_id','name','category_name','brand_name','item_description'],axis=1)
print data1.columns
#train, test = train_test_split(data, test_size=0.7)
#print train.columns

ValueError: labels [u'Cat1' u'Cat2' u'Cat3' 'category_name'] not contained in axis

In [ ]:
len(data[data.Cat1.isnull()==True])

In [18]:
trn_columns = ['median_item_condition_id1','shipping','len_name','brand_nameTf', u'Cat1Tf',
       u'Cat2Tf', u'Cat3Tf']
targ = ['log_price']
trn_columns = ['len_name','mean_it_id','mean_shipping',u'mean_brand_name',u'mean_Cat1', u'mean_Cat2', u'mean_Cat3']

In [19]:
train,test = train_test_split(data,test_size=0.3)

In [20]:
del data1

NameError: name 'data1' is not defined

### LinearRegression

In [21]:
lr=LinearRegression()
lr.fit(train[trn_columns],train[targ])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [22]:
preds = lr.predict(test[trn_columns])
print len(preds)

444761


In [23]:
np.sqrt(np.mean(np.square(preds-test[targ])))

log_price    0.596196
dtype: float64

### Ridge

In [24]:
from sklearn.linear_model import Ridge
rg=Ridge()
rg.fit(train[trn_columns],train[targ])

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [25]:
preds = rg.predict(test[trn_columns])
print len(preds)

222381


In [26]:
np.sqrt(np.mean(np.square(preds-test[targ])))

log_price    0.599638
dtype: float64

### SVR

In [ ]:
svr = LinearSVR()
svr.fit(train[trn_columns],train[targ])

In [ ]:
preds = svr.predict(test[trn_columns])
print len(preds)

In [ ]:
df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))

In [ ]:
svrr = SVR()
svrr.fit(train[trn_columns],train[targ])

In [ ]:
preds = svrr.predict(test[trn_columns])
print len(preds)

In [ ]:
df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))

### RandomForest

In [24]:
gbr=GradientBoostingRegressor()
gbr.fit(train[trn_columns],train[targ])

C:\Python27\lib\site-packages\sklearn\utils\validation.py:547: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [25]:
preds = gbr.predict(test[trn_columns])

df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))


0.576209799651


In [29]:
import gc
gc.collect()
print np.sqrt(np.mean(np.square(preds-test[targ])))

ValueError: iterator is too large

### Adaboost

In [ ]:
abr=AdaBoostRegressor()
abr.fit(train[trn_columns],train[targ])

In [ ]:
preds = abr.predict(test[trn_columns])

In [ ]:
df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))

###  SGD

In [ ]:
sgd = SGDRegressor( n_iter=100)
sgd.fit(train[trn_columns],train[targ])

In [ ]:
preds = sgd.predict(test[trn_columns])
print preds

In [ ]:
df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))

### MLP

In [29]:
mlp = MLPRegressor()
mlp.fit(train[trn_columns],train[targ])

C:\Python27\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [30]:
preds = mlp.predict(test[trn_columns])

In [31]:
df=pd.DataFrame()
df['preds']=preds
df['price']=list(test[targ]['log_price'])
print np.sqrt(np.mean(np.square(df.preds-df.price)))

0.597789913981
